## Performance de uma carteira de ativos

###### Importando as bibliotecas

In [ ]:
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf

###### Importando os dados

In [ ]:
carteira = pd.read_excel('carteira.xlsx')
display(carteira)

###### Criando nosso Dataframe de Cotações dos ativos da carteira

In [ ]:
cotacoes_carteira = pd.DataFrame()

for ativo in carteira['Ativos']:
  cotacoes_carteira[ativo] = yf.download('{}.SA'.format(ativo), start='2020-01-01', end='2020-11-10', progress=False)['Adj Close']
  
display(cotacoes_carteira)

###### Validando as informações geradas

In [ ]:
cotacoes_carteira.info()

###### Ajustando os dados validados que estão como nulos

In [ ]:
df_media = cotacoes_carteira.mean()
# cotacoes_carteira = cotacoes_carteira.fillna(df_media)
cotacoes_carteira = cotacoes_carteira.ffill()

display(cotacoes_carteira)

###### Analisando a performance da carteira de forma individual para normalizar os dados

In [ ]:
carteira_norm = cotacoes_carteira / cotacoes_carteira.iloc[0]
carteira_norm.plot(figsize=(15,5))
plt.legend(loc='upper left')

###### Buscando o Ibovespa para comparação

In [ ]:
cotacoes_ibov = yf.download('^BVSP', start='2020-01-01', end='2020-11-10', progress=False)
display(cotacoes_ibov)

###### Criando um Dataframe da Carteira com as quantidades de ações

In [ ]:
valor_investido = pd.DataFrame()

for ativo in carteira['Ativos']:
  valor_investido[ativo] = cotacoes_carteira[ativo] * carteira.loc[carteira['Ativos'] == ativo, 'Qtde'].values[0]

# Remover GNDI3 pois não tem cotação
valor_investido = valor_investido.drop('GNDI3', axis=1)

display(valor_investido)

###### Carteira x IBOV

In [ ]:
valor_investido['Total'] = valor_investido.sum(axis=1)

valor_investido_norm = valor_investido / valor_investido.iloc[0]
cotacoes_ibov_norm = cotacoes_ibov / cotacoes_ibov.iloc[0]

valor_investido_norm['Total'].plot(figsize=(15,5), label='Cateira')
cotacoes_ibov_norm['Adj Close'].plot(figsize=(15,5), label='IBOV')
plt.legend()
plt.show()

In [ ]:
retorno_carteira = valor_investido['Total'][-1] / valor_investido['Total'][0]
retorno_ibov = cotacoes_ibov['Adj Close'][-1] / cotacoes_ibov['Adj Close'][0]
print('Retorno carteira: {:.2f}%'.format(retorno_carteira))
print('Retorno IBOV: {:.2f}%'.format(retorno_ibov))

###### Correlação da Carteira com o IBOV

In [ ]:
correlacao = valor_investido['Total'].corr(cotacoes_ibov['Adj Close'])
print('Correlação: {:.2f}%'.format(correlacao))